**2. Working with ensemble data**

To do this you will need to download "HadSST.4.0.0.0_ensemble.zip" from https://www.metoffice.gov.uk/hadobs/hadsst4/data/download.html it is a massive folder of 200 ensemble members! A useful tool for looking at netcdf files: Panoply (Nasa) https://www.giss.nasa.gov/tools/panoply/ 

Model ensembles are very common. They represent a group (anywhere between 2 and 100+) of *model runs* using similar initial conditions. The initial conditions are varied slightly for each run to mimic randomness within the earth system. 

1. Initialise any arrays

2. Create a for loop which loops through the files in the ensemble foler and loads each sst file and the variables

3. Within the for loop calculate the baseline, current climates and anomalies for each of the files
uses formulas from last week 

4. For each type of climate: baselines, currents and anomalies, concatenate the arrays so you end up with three arrays of shape (200,36,72) (as you have 200 files, you will have 200 values for each grid point) 
data2 = pd.concat([year, anomaly], axis=1) # learned to concatenate two series into a df

5. outside of the for loop calculate the anomaly mean (for each grid point)



6. Now loop through all of the grid points and compare the distributions of the baseline compared to the currents - are they significantly different (i.e. is the global sst now significantly different from what it was during the baseline?)


You should output:

* an anomaly array of shape (36,72) which represents current - baseline SST
* a boolean array of shape (36,72) which represents whether there is a statistical significant difference between baseline and current
So this is not straightforward, if you're finding it impossible then just use the answers for this bit and try to understand what each line is doing.



In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
from netCDF4 import Dataset,num2date
from datetime import date, timedelta

**1. Initialise any arrays** 

In [34]:
bas_a = np.zeros((1,36,72)) # made this three-dimensional. What happens if we set this to be zero?
cur_a = np.zeros((1,36,72))

**2. Create a for loop which loops through the files in the ensemble folder and loads each sst file and the variables, and calculates the baseline, current climates, and anomalies for each of the files.**

In [12]:
# Setting up variables (outside of loop)
example = Dataset('/Users/bendixon/Documents/HadSST/HadSST.4.0.0.0_ensemble_member_1.nc')

# time and lat and lon are global variables
latitude = example.variables['latitude'][:]
longitude = example.variables['longitude'][:]

# time 
time = example.variables['time'][:]
time_units = example.variables['time'].units
calendar = example.variables['time'].calendar # I think Gregorian might be the default, not sure if needed
time_convert = num2date(time, time_units, calendar)
i_1950 = 1200; i_1980 = 1569; i_2000 = 1800; i_2018 = 2027    

In [26]:
def baseline(fn):
    # Opening the file and loading the temperature value
    swta = Dataset(fn).variables['tos'][:]
    swta = np.ma.masked_values(swta, 9.969209968386869e+36)
    # Setting the time parameters () 
    baseline = swta[i_1950:i_1980,:,:]
    baseline = np.mean(baseline,axis=0) 
    return baseline # Returning the baseline value from the loop

In [27]:
def current(fn):
    # Opening the file and loading the temperature value
    swta = Dataset(fn).variables['tos'][:]
    swta = np.ma.masked_values(swta, 9.969209968386869e+36)
    # Setting the time parameters () 
    current = swta[i_2000:i_2018,:,:]
    current = np.mean(current,axis=0)
    return current # Returning the baseline value from the loop

*Potential improvements*

* Merge baseline and current files to do mean(start_date,end_date) so no need for individual functions
* Also work out how to go from start and end date in DD-MM-YYYY to the position in the array

In [35]:
directory = '/Users/bendixon/Documents/HadSST/'
os.chdir('/Users/bendixon/Documents/HadSST/') # do we need to do both of these?
# looping through the models, working out the baseline
for filename in os.listdir(directory):
    bas = np.array([baseline(filename)])
    bas_b = np.concatenate((bas,bas_a),axis=0)
    bas_a = bas_b # Maybe there's a better way of doing this another loop
print(bas_b.shape)
# looping through the models, working out the current value
for filename in os.listdir(directory):
    cur = np.array([current(filename)])
    cur_b = np.concatenate((cur,cur_a),axis=0)
    cur_a = cur_b
print(cur_b.shape)
# I'd expect it's more efficient to do cur_b - bas_b rather than opening each file 

(201, 36, 72)
(201, 36, 72)


In [36]:
anom_a = np.zeros((201,36,72))
anom_a = np.subtract(cur_b,bas_b)
print(anom_a.shape)

(201, 36, 72)


## Workings/ideas - ignore the below

In [ ]:
# initialise empty arrays

# open data in for loop
for i in range(1,200):

    # calculate baseline and current global mean sst
    
    # you can't np.concatenate to an empty array so create an if statement to deal with the first instance    


# calculate ensemble mean sst anomaly for each grid point

# loop through each grid point and assess whether the current values are significantly different from the baseline
for i in range(36):
    for j in range(72):
        # ...     

anomaly = 
significance =

In [ ]:
# Workings - experimenting with .insert to populate the array
b = [1,1,1,1]
for i in range(len(b)):
    b.insert(99999,5)
print(b)

In [ ]:
# I think the action here is to use a loop to fill in an array
# could try this out by making a loop to fill another array
a = np.zeros((4,20))
for i in range(1,20):
    a[0,i] = 5 # this works! could also use .insert? or np.concatenate?

In [ ]:
# print current working directory
os.getcwd()

In [11]:
a = np.array([[1, 2, 3],[4, 5, 6]])
b = np.array([[10, 20, 30],[40, 50, 60]])
c = np.concatenate((a,b),axis=0) # 
print(c.shape)
print(c)

(4, 3)
[[ 1  2  3]
 [ 4  5  6]
 [10 20 30]
 [40 50 60]]


In [ ]:
# old function definition, in case needed later
def anomaly(fn):
    # Opening the file and loading the temperature value
    swta_file = Dataset(fn)
    variables = swta_file.variables
    swta = swta_file.variables['tos'][:]
    swta = np.ma.masked_values(swta, 9.969209968386869e+36)
    # Setting the time parameters () 
    baseline = swta[i_1950:i_1980,:,:]
    baseline = np.mean(baseline,axis=0) 
    current = swta[i_2000:i_2018,:,:]
    current = np.mean(current,axis=0)
    # Printing the anomaly
    anom = current - baseline
    anom = np.array(anom, dtype=float)
    return anom

In [ ]:
print(time_convert[1200])
print(time_convert[1560]) # not sure why some months there are measurements at 00:00 or 12:00
print(time_convert[1800])
print(time_convert[2027]) # this is only up to 2018, but that's the size of the array